In [2]:
import os
import numpy as np
import zipfile
import requests
from tqdm import tqdm
import torch
import torch.nn as nn
# import torch.nn.functional as F
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from sklearn.model_selection import train_test_split
from torch import optim
from pathlib import Path
import json
import wandb
import time
import copy

In [3]:
dataset_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
dataset_zip_path = "/kaggle/working/nature_12K.zip"
dataset_dir = "nature_12K"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
num_classes = 10  # For iNaturalist (adjust if needed)
batch_size = 16
num_epochs = 10
learning_rate = 1e-3

In [4]:
if not os.path.exists(dataset_dir):
    if not os.path.exists(dataset_zip_path):
        print("Downloading iNaturalist-12K...")
        response = requests.get(dataset_url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(dataset_zip_path, 'wb') as f, tqdm(
            desc=dataset_zip_path,
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                size = f.write(data)
                bar.update(size)

/kaggle/working/nature_12K.zip: 100%|██████████| 3.55G/3.55G [02:02<00:00, 31.1MiB/s] 


In [5]:
if not os.path.exists(dataset_dir):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(".")

Extracting dataset...


In [6]:
def prepare_datasets(data_dir, val_split=0.2, batch_size=32, image_size=(224, 224)):
    data_dir = Path(data_dir)

    # Define transforms (customize as needed)
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),  # Converts to [0, 1] and CxHxW
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),  # Normalize RGB
    ])

    # Load training and testing datasets
    full_train_dataset = ImageFolder(root=data_dir / "train", transform=transform)
    test_dataset = ImageFolder(root=data_dir / "val", transform=transform)

    # Create validation split from training set
    val_size = int(val_split * len(full_train_dataset))
    train_size = len(full_train_dataset) - val_size

    train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

    # Extract X and Y by iterating over DataLoader batches if needed
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # # Convert to X, Y tensors
    # def extract_XY(loader):
    #     for X, Y in loader:
    #         return X, Y

    # X_train, Y_train = extract_XY(train_loader)
    # X_val, Y_val = extract_XY(val_loader)
    # X_test, Y_test = extract_XY(test_loader)

    return train_loader, val_loader, test_loader

In [7]:
train_loader, val_loader, test_loader = prepare_datasets(
    data_dir='/kaggle/working/inaturalist_12K', val_split=0.2, batch_size=64, image_size=(224, 224)
)

In [14]:
api_key = "7040d84a3ed65a967eb3389dd6fe774b418576ed" 
wandb.login(key=api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
def train(config=None):
    # Initialize wandb
    with wandb.init(config=config):
        config = wandb.config
        
        # Initialize best_val_acc at the beginning of the function
        best_val_acc = 0.0

# Set random seed for reproducibility
        torch.manual_seed(config.seed)
        np.random.seed(config.seed)

        # Data augmentation and normalization based on config
        if config.use_augmentation:
            train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.2, contrast=0.2),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
            ])
        else:
            train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
            ])

        val_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])

        # Load dataset
        train_dataset = datasets.ImageFolder(
            root='/kaggle/working/inaturalist_12K/train',  # Update to match your dataset path
            transform=train_transform
        )
        
        # Split into train and validation
        train_indices, val_indices = train_test_split(
            list(range(len(train_dataset))),
            test_size=0.2,
            stratify=train_dataset.targets,
            random_state=config.seed
        )
        
        train_subset = torch.utils.data.Subset(train_dataset, train_indices)
        val_subset = torch.utils.data.Subset(train_dataset, val_indices)
        
        train_loader = DataLoader(
            train_subset,
            batch_size=config.batch_size,
            shuffle=True,
            num_workers=4
        )
        
        val_loader = DataLoader(
            val_subset,
            batch_size=config.batch_size,
            shuffle=False,
            num_workers=4
        )

        model = models.resnet50(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)  # Replace last layer

        # Freeze all layers
        for param in model.parameters():
            param.requires_grad = False
        
        # Replace the final layer (fc) for 10-class classification
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)
        
        # Only the final layer's parameters will be updated
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.fc.parameters(), lr=config.learning_rate)

        # The rest of the training code remains the same
        for epoch in range(config.epochs):
            # Training phase
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            for batch_idx, (inputs, targets) in enumerate(train_loader):
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                _, predicted = outputs.max(1)
                train_total += targets.size(0)
                train_correct += predicted.eq(targets).sum().item()

                if batch_idx % 100 == 0:
                    print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

            train_acc = 100. * train_correct / train_total

            # Validation phase
            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)

                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    val_total += targets.size(0)
                    val_correct += predicted.eq(targets).sum().item()

            val_acc = 100. * val_correct / val_total

            # Log metrics to wandb
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss / len(train_loader),
                "train_acc": train_acc,
                "val_loss": val_loss / len(val_loader),
                "val_acc": val_acc
            })

            # Save best model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), 'best_model.pth')
                print(f"Best model saved with val_acc: {val_acc:.2f}% at epoch {epoch}")

            print(f'Epoch: {epoch}, Train Loss: {train_loss/len(train_loader):.4f}, '
                  f'Train Acc: {train_acc:.2f}%, Val Loss: {val_loss/len(val_loader):.4f}, '
                  f'Val Acc: {val_acc:.2f}%')
        

In [10]:
# Updated sweep configuration
sweep_config = {
    'method': 'random',  # or 'grid' or 'bayes'
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-5,
            'max': 1e-3
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'epochs': {
            'value': 5
        },
        'seed': {
            'values': [42, 2023, 7]
        },
        'use_augmentation': {
            'values': [True, False]
        },
        'trainable_layers': {
            'values': [1, 2, 3, 5, 7]  # Train different numbers of layers from the end
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="da6401_partB_01")  # Replace with your wandb project name
# Run the sweep 50 times
wandb.agent(sweep_id, function=train, count=50)

Create sweep with ID: 7uxv9ux3
Sweep URL: https://wandb.ai/da24m004-iitmaana/da6401_partB_01/sweeps/7uxv9ux3


wandb: Agent Starting Run: azmcymsl with config:
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0006949258369787645
wandb: 	seed: 7
wandb: 	trainable_layers: 5
wandb: 	use_augmentation: False
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 205MB/s]


Epoch: 0, Batch: 0, Loss: 2.4081
Epoch: 0, Batch: 100, Loss: 1.2058
Best model saved with val_acc: 70.45% at epoch 0
Epoch: 0, Train Loss: 1.2552, Train Acc: 62.60%, Val Loss: 0.9108, Val Acc: 70.45%
Epoch: 1, Batch: 0, Loss: 0.7738
Epoch: 1, Batch: 100, Loss: 0.7431
Best model saved with val_acc: 71.75% at epoch 1
Epoch: 1, Train Loss: 0.8432, Train Acc: 73.30%, Val Loss: 0.8489, Val Acc: 71.75%
Epoch: 2, Batch: 0, Loss: 0.7596
Epoch: 2, Batch: 100, Loss: 0.7352
Best model saved with val_acc: 73.85% at epoch 2
Epoch: 2, Train Loss: 0.7815, Train Acc: 75.08%, Val Loss: 0.7995, Val Acc: 73.85%
Epoch: 3, Batch: 0, Loss: 0.7122
Epoch: 3, Batch: 100, Loss: 0.4950
Epoch: 3, Train Loss: 0.7242, Train Acc: 76.38%, Val Loss: 0.7941, Val Acc: 73.65%
Epoch: 4, Batch: 0, Loss: 0.8440
Epoch: 4, Batch: 100, Loss: 0.5876
Best model saved with val_acc: 74.00% at epoch 4
Epoch: 4, Train Loss: 0.7073, Train Acc: 77.20%, Val Loss: 0.7985, Val Acc: 74.00%


epoch,▁▃▅▆█
train_acc,▁▆▇██
train_loss,█▃▂▁▁
val_acc,▁▄█▇█
val_loss,█▄▁▁▁
epoch,4
train_acc,77.19715
train_loss,0.70732
val_acc,74
val_loss,0.79854


...

epoch,▁▃▅▆█
train_acc,▁▆▇██
train_loss,█▄▂▁▁
val_acc,▁▄▆▇█
val_loss,█▄▂▁▁
epoch,4
train_acc,73.17165
train_loss,0.8941
val_acc,73.5
val_loss,0.88106


wandb: Agent Starting Run: 1yrjddoo with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0008536756265458657
wandb: 	seed: 42
wandb: 	trainable_layers: 5
wandb: 	use_augmentation: True


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 0, Batch: 0, Loss: 2.4731
Epoch: 0, Batch: 100, Loss: 1.3195
Epoch: 0, Batch: 200, Loss: 0.7973
Best model saved with val_acc: 69.05% at epoch 0
Epoch: 0, Train Loss: 1.2553, Train Acc: 59.53%, Val Loss: 0.9311, Val Acc: 69.05%
Epoch: 1, Batch: 0, Loss: 0.9858
Epoch: 1, Batch: 100, Loss: 1.1458
Epoch: 1, Batch: 200, Loss: 0.8105
Best model saved with val_acc: 71.25% at epoch 1
Epoch: 1, Train Loss: 0.9582, Train Acc: 68.13%, Val Loss: 0.8703, Val Acc: 71.25%
Epoch: 2, Batch: 0, Loss: 0.8693
Epoch: 2, Batch: 100, Loss: 0.7910
Epoch: 2, Batch: 200, Loss: 0.9000
Best model saved with val_acc: 71.65% at epoch 2
Epoch: 2, Train Loss: 0.8990, Train Acc: 70.21%, Val Loss: 0.8511, Val Acc: 71.65%
Epoch: 3, Batch: 0, Loss: 0.8415
Epoch: 3, Batch: 100, Loss: 0.9200
Epoch: 3, Batch: 200, Loss: 1.3347
Best model saved with val_acc: 73.50% at epoch 3
Epoch: 3, Train Loss: 0.8654, Train Acc: 71.28%, Val Loss: 0.8028, Val Acc: 73.50%
Epoch: 4, Batch: 0, Loss: 0.6301
Epoch: 4, Batch: 100, Loss:

epoch,▁▃▅▆█
train_acc,▁▆▇██
train_loss,█▃▂▁▁
val_acc,▁▄▄▇█
val_loss,█▅▄▁▁
epoch,4
train_acc,71.64646
train_loss,0.8546
val_acc,74.3
val_loss,0.79771


In [15]:
sweep_id

'da24m004-iitmaana/da6401_partB/sweeps/7uxv9ux3'

In [16]:
import wandb
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Define test transform
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load test dataset
test_dataset = datasets.ImageFolder(
    root='/kaggle/working/inaturalist_12K/val',  # <-- Update path if needed
    transform=test_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,  # Use a reasonable batch size
    shuffle=False,
    num_workers=0   # Avoid BrokenPipeError
)

# Fetch the best run from W&B sweep
api = wandb.Api()
sweep_id = f"{sweep_id}"  # <-- Replace this with your actual sweep_id
sweep = api.sweep(sweep_id)

# Sort runs by highest validation accuracy
best_run = sorted(sweep.runs, key=lambda run: run.summary.get("val_acc", 0), reverse=True)[0]
best_config = best_run.config
print("Best Run ID:", best_run.id)
print("Best Val Accuracy:", best_run.summary["val_acc"])
print("Best Config:", best_config)

# Get the trainable_layers parameter from the best config
trainable_layers = best_config.get('trainable_layers', 1)  # Default to 1 if not found

# Determine the correct model file to download based on the best config
# The model filename now includes the number of trainable layers
model_filename = f"best_model_layers{trainable_layers}.pth"
try:
    # Try to download the model with the new naming convention
    model_path = best_run.file(model_filename).download(exist_ok=True).name
    print(f"Downloaded model: {model_filename}")
except Exception as e:
    # Fall back to the default name if the specific file isn't found
    print(f"Could not find {model_filename}, falling back to default name.")
    model_path = best_run.file("best_model.pth").download(exist_ok=True).name
    print(f"Downloaded model: best_model.pth")

# Re-instantiate model with the same architecture as during training
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Initially freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# If needed for evaluation, unfreeze the same layers that were trained
# This isn't strictly necessary for inference, but ensures the model setup matches training
if trainable_layers > 1:  # If more than just the FC layer was trained
    # Get a list of all model layers
    layers = [
        model.conv1, model.bn1, model.relu, model.maxpool,  # Initial layers
        model.layer1,  # Layer 1
        model.layer2,  # Layer 2
        model.layer3,  # Layer 3
        model.layer4,  # Layer 4
        model.avgpool,  # Average pooling
        model.fc      # Final fully connected layer
    ]
    
    # Unfreeze the last k layers as in training
    trainable_layers_list = layers[-trainable_layers:]
    for layer in trainable_layers_list:
        for param in layer.parameters():
            param.requires_grad = True
    
    print(f"Model configured with {trainable_layers} trainable layers from the end")

# Load the saved model weights
model.load_state_dict(torch.load(model_path))
model.eval()

# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {device}")

# Evaluate on test set
def evaluate_on_test(model, loader):
    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    
    class_correct = [0] * 10
    class_total = [0] * 10
    
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            # Per-class accuracy
            for i in range(targets.size(0)):
                label = targets[i]
                pred = predicted[i]
                if label == pred:
                    class_correct[label] += 1
                class_total[label] += 1
    
    # Overall accuracy
    test_acc = 100. * correct / total
    print(f"Test Accuracy: {test_acc:.2f}%")
    
    # Per-class accuracy
    print("Per-class accuracy:")
    for i in range(10):
        class_acc = 100. * class_correct[i] / class_total[i] if class_total[i] > 0 else 0
        print(f"Class {i}: {class_acc:.2f}%")
    
    # Log to wandb if desired
    if wandb.run is not None:
        wandb.log({
            "test_accuracy": test_acc,
            "per_class_accuracy": {f"class_{i}": 100. * class_correct[i] / class_total[i] 
                                  if class_total[i] > 0 else 0 for i in range(10)}
        })
    
    return test_acc, class_correct, class_total

# Run the evaluation
print("\nEvaluating model on test set...")
test_acc, class_correct, class_total = evaluate_on_test(model, test_loader)

# Print a summary
print("\nModel Summary:")
print(f"Best Run ID: {best_run.id}")
print(f"Number of trainable layers: {trainable_layers}")
print(f"Best validation accuracy: {best_run.summary['val_acc']:.2f}%")
print(f"Test accuracy: {test_acc:.2f}%")

# Optional: Additional analysis
try:
    # If you want to create a confusion matrix or more detailed analysis
    import matplotlib.pyplot as plt
    import numpy as np
    from sklearn.metrics import confusion_matrix
    
    # Function to get all predictions for confusion matrix
    def get_all_predictions(model, loader):
        all_targets = []
        all_preds = []
        model.eval()
        with torch.no_grad():
            for inputs, targets in loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                all_targets.extend(targets.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())
        return np.array(all_targets), np.array(all_preds)
    
    # Get predictions
    targets, predictions = get_all_predictions(model, test_loader)
    
    # Create confusion matrix
    cm = confusion_matrix(targets, predictions)
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    classes = [str(i) for i in range(10)]
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    # Add text annotations
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
    # Save the confusion matrix
    plt.savefig('confusion_matrix.png')
    
    # Log to wandb if desired
    if wandb.run is not None:
        wandb.log({"confusion_matrix": wandb.Image('confusion_matrix.png')})
    
    print("Confusion matrix saved as 'confusion_matrix.png'")
except Exception as e:
    print(f"Could not create confusion matrix: {e}")

CommError: Could not find sweep <Sweep da24m004-iitmaana/da6401_partB/7uxv9ux3 (Unknown State)>